In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency

class CHAIDNode:
    def __init__(self, data, target, depth=0, max_depth=None, min_samples_split=20):
        self.data = data
        self.target = target
        self.depth = depth
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.children = []
        self.split_feature = None
        self.split_value = None
        self.is_leaf = False
        self.class_label = None
        self._split()

    def _split(self):
        if (self.max_depth is not None and self.depth >= self.max_depth) or \
           len(self.data) < self.min_samples_split:
            self.is_leaf = True
            self.class_label = self.data[self.target].mode()[0]
            return

        features = [col for col in self.data.columns if col != self.target]
        best_chi2 = 0
        best_feature = None
        best_groups = None
        for feature in features:
            groups = self.data.groupby(feature)
            contingency_table = []
            for _, group in groups:
                contingency_table.append(group[self.target].value_counts())
            contingency_table = pd.DataFrame(contingency_table).fillna(0).values
            if contingency_table.size and contingency_table.shape[0] > 1:
                chi2, p, _, _ = chi2_contingency(contingency_table)
                if chi2 > best_chi2:
                    best_chi2 = chi2
                    best_feature = feature
                    best_groups = [group for _, group in groups]

        if best_feature is not None and len(best_groups) > 1:
            self.split_feature = best_feature
            for group in best_groups:
                child = CHAIDNode(
                    group,
                    self.target,
                    depth=self.depth+1,
                    max_depth=self.max_depth,
                    min_samples_split=self.min_samples_split
                )
                self.children.append(child)
        else:
            self.is_leaf = True
            self.class_label = self.data[self.target].mode()[0]

    def predict(self, x):
        if self.is_leaf or self.split_feature not in x:
            return self.class_label
        for child in self.children:
            if child.data.iloc[0][self.split_feature] == x[self.split_feature]:
                return child.predict(x)
        return self.class_label

# Example DataFrame based on MODIS values
df = pd.DataFrame({
    'NDVI': [2011, 1721, 1977, 1961, 1816, 2944, 1679, 1972, 1478, 1666],
    'EVI': [1330, 1160, 1315, 1225, 1133, 2132, 1107, 1232, 910, 1072],
    'Red': [1438, 1876, 1544, 1422, 1496, 1429, 1635, 1323, 1501, 1463],
    'NIR': [2162, 2656, 2305, 2116, 2160, 2622, 2295, 1973, 2022, 2048],
    'MIR': [1546, 1541, 1541, 1444, 1444, 1608, 1608, 1540, 1540, 1546],
    'pixelreliability': [1,1,1,1,1,1,1,1,1,1],
    'health': ['Good','Poor','Good','Good','Fair','Good','Poor','Fair','Good','Poor'] # Example categorical target
})

# Use "health" as target (categorical, for CHAID)
tree = CHAIDNode(df, target='health', max_depth=3, min_samples_split=2)

# Predict an instance (example)
instance = df.iloc[0].to_dict()
prediction = tree.predict(instance)
print('Predicted class:', prediction)

Predicted class: Good


In [ ]:
# Step 1: Import required libraries
import pandas as pd
import json

# Step 2: Load JSON into DataFrame
# Place response.json in your working directory or provide the full path
with open('response.json', 'r') as file:
    data = json.load(file)

# Usually, JSON from MODIS-like APIs has a data section with rows, or sometimes "features"
# Inspect the structure for the precise path to tabular data
# Here's a generic way to flatten a dict/list style JSON:
# Adjust ['data'] to match the key in your file - corrected based on error
# Assuming the data is within 'modis' -> 'interpreted_data' and is a list of records
if 'modis' in data and 'interpreted_data' in data['modis']:
    df = pd.DataFrame(data['modis']['interpreted_data']) # Adjust based on the actual structure
    print(df.head()) # See columns and first rows
else:
    print("Could not find the expected data structure in the JSON.")
    df = None # Set df to None to avoid further errors

# Step 3: Use with CHAID model code
if df is not None:
  target_col = 'vegetation_health' # Change to your categorical target (e.g. 'health', 'risk', etc.)

  if target_col in df.columns:
    tree = CHAIDNode(df, target=target_col, max_depth=3, min_samples_split=20)

    # Step 4: Predict with an instance
    # Ensure the instance has the required columns for prediction
    if not df.empty:
        instance = df.iloc[0].to_dict()
        prediction = tree.predict(instance)
        print('Predicted class:', prediction)
    else:
        print("DataFrame is empty, cannot make a prediction.")
  else:
    print(f"Target column '{target_col}' not found in the DataFrame.")

                      coordinate_specific area_summary visualization  \
ndvi_latest                        0.1754          NaN          None   
evi_latest                           None          NaN          None   
vegetation_health                STRESSED          NaN          None   
fire_risk_vegetation                 HIGH          NaN          None   
last_observation_date          2025-07-28          NaN          None   

                      risk_assessment  
ndvi_latest                      HIGH  
evi_latest                       HIGH  
vegetation_health                HIGH  
fire_risk_vegetation             HIGH  
last_observation_date            HIGH  
Target column 'vegetation_health' not found in the DataFrame.


In [ ]:
print(data.keys())


dict_keys(['request_id', 'event_id', 'location', 'timestamp', 'summary', 'total_processing_time_ms', 'modis', 'weather', 'topography'])


In [ ]:
!pip install CHAID

In [ ]:
# ===============================================
# CHAID Model Implementation - Final Corrected Version
# ===============================================

# Step 1: Import required libraries
import pandas as pd
import json
from CHAID import Tree

# -------------------------------------------------------
# Step 2: Load JSON and Normalize into a Flat DataFrame
# -------------------------------------------------------
with open("response.json", "r") as file:
    data = json.load(file)

# --- Handle deeply nested JSON automatically ---
def flatten_json(y):
    """Recursively flattens nested JSON into a single dict."""
    out = {}

    def flatten(x, name=''):
        if isinstance(x, dict):
            for a in x:
                flatten(x[a], f"{name}{a}_")
        elif isinstance(x, list):
            i = 0
            for a in x:
                flatten(a, f"{name}{i}_")
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

# Flatten entire JSON
flat_data = flatten_json(data)
df = pd.DataFrame([flat_data])

print("\n✅ Flattened JSON successfully converted to DataFrame!")
print(df.head(3))
print("\nColumns available:", len(df.columns))

# -------------------------------------------------------
# Step 3: Identify the target column automatically
# -------------------------------------------------------
# Find any column that includes 'vegetation_health'
possible_targets = [col for col in df.columns if 'vegetation_health' in col.lower()]

if not possible_targets:
    raise ValueError("❌ Could not find any column containing 'vegetation_health'.")

target_col = possible_targets[0]
print(f"\n✅ Using target column: {target_col}")

# -------------------------------------------------------
# Step 4: Prepare data for CHAID
# -------------------------------------------------------
df = df.dropna(subset=[target_col])
df = df.fillna("Unknown")

predictors = [col for col in df.columns if col != target_col]
print(f"✅ Predictor columns count: {len(predictors)}")

# -------------------------------------------------------
# Step 5: Build CHAID model
# -------------------------------------------------------
try:
    tree = Tree.from_pandas_df(df, {}, target_col, max_depth=3, min_child_node_size=1)
    print("\n🌳 CHAID Decision Tree:")
    tree.print_tree()

except Exception as e:
    print("❌ Error while building CHAID tree:", e)

# -------------------------------------------------------
# Step 6: Save CHAID tree output safely (text format)
# -------------------------------------------------------
try:
    import io
    buffer = io.StringIO()
    tree.print_tree(out=buffer)
    tree_text = buffer.getvalue()

    with open("chaid_tree_output.txt", "w") as f:
        f.write(tree_text)

    print("\n✅ CHAID tree successfully saved to 'chaid_tree_output.txt'")

except Exception as e:
    print("⚠️ Could not save CHAID tree output:", e)

# -------------------------------------------------------
# Step 7: Simple Prediction Example (optional)
# -------------------------------------------------------
try:
    instance = df.iloc[0].to_dict()
    prediction = tree.predict(instance)
    print("\n🔮 Predicted class for first instance:", prediction)
except Exception as e:
    print("⚠️ Prediction skipped due to:", e)



✅ Flattened JSON successfully converted to DataFrame!
         request_id event_id  location_latitude  location_longitude  \
0  req_2dc79f9a3fcd     None            34.0522           -118.2437   

   location_buffer_meters location_coordinate_system  \
0                    1000                  EPSG:4326   

                    timestamp  summary_total_sources  \
0  2025-09-19T20:37:11.082935                      4   

   summary_successful_sources  summary_total_errors  ...  \
0                           3                     1  ...   

   topography_interpreted_data_visualization_resolution_meters  \
0                                               30.0             

  topography_interpreted_data_visualization_color_scheme  \
0                                               None       

   topography_interpreted_data_risk_assessment  \
0                                          LOW   

  topography_metadata_processing_time_ms topography_metadata_data_currency  \
0                     

In [ ]:
# ===============================================
# CHAID Model Implementation (CHAID v5.4.2 Compatible)
# ===============================================

import pandas as pd
import json
from CHAID import Tree

# -------------------------------------------------------
# Step 1: Load and flatten JSON file
# -------------------------------------------------------
with open("response.json", "r") as file:
    data = json.load(file)

def flatten_json(y):
    """Recursively flattens nested JSON."""
    out = {}
    def flatten(x, name=''):
        if isinstance(x, dict):
            for a in x:
                flatten(x[a], f"{name}{a}_")
        elif isinstance(x, list):
            i = 0
            for a in x:
                flatten(a, f"{name}{i}_")
                i += 1
        else:
            out[name[:-1]] = x
    flatten(y)
    return out

flat_data = flatten_json(data)
df = pd.DataFrame([flat_data])

print("\n✅ Flattened JSON successfully converted to DataFrame!")
print(df.head(3))
print("\nColumns available:", len(df.columns))

# -------------------------------------------------------
# Step 2: Find the target column
# -------------------------------------------------------
possible_targets = [col for col in df.columns if "vegetation_health" in col.lower()]

if not possible_targets:
    raise ValueError("❌ Could not find any column containing 'vegetation_health'.")
target_col = possible_targets[0]
print(f"\n✅ Using target column: {target_col}")

# -------------------------------------------------------
# Step 3: Clean data
# -------------------------------------------------------
df = df.fillna("Unknown")
unique_classes = df[target_col].unique()
print(f"✅ Unique classes in target: {unique_classes}")

if len(unique_classes) < 2:
    print("⚠️ Warning: Only one target class detected. CHAID cannot split meaningfully.")

predictors = [col for col in df.columns if col != target_col]
print(f"✅ Predictor columns count: {len(predictors)}")

# -------------------------------------------------------
# Step 4: Build CHAID model
# -------------------------------------------------------
try:
    tree = Tree.from_pandas_df(df, {}, target_col, max_depth=3, min_child_node_size=1)
    print("\n🌳 CHAID Decision Tree:")
    tree.print_tree()
except Exception as e:
    print("❌ Error while building CHAID tree:", e)

# -------------------------------------------------------
# Step 5: Save the CHAID tree output (manual redirect)
# -------------------------------------------------------
try:
    import sys
    original_stdout = sys.stdout
    with open("chaid_tree_output.txt", "w") as f:
        sys.stdout = f
        tree.print_tree()
        sys.stdout = original_stdout
    print("\n✅ CHAID tree successfully saved as 'chaid_tree_output.txt'")
except Exception as e:
    print("⚠️ Could not save CHAID tree:", e)

# -------------------------------------------------------
# Step 6: Pseudo-Prediction Demonstration (since CHAID isn’t predictive)
# -------------------------------------------------------
try:
    sample = df.iloc[0].to_dict()
    print("\n🔎 Example record:")
    print(sample)

    # Just show which leaf category this record falls under (conceptual demo)
    print("\n⚠️ Note: CHAID doesn’t have .predict(); use DecisionTreeClassifier if predictive behavior is required.")
except Exception as e:
    print("⚠️ Could not show example record:", e)



✅ Flattened JSON successfully converted to DataFrame!
         request_id event_id  location_latitude  location_longitude  \
0  req_2dc79f9a3fcd     None            34.0522           -118.2437   

   location_buffer_meters location_coordinate_system  \
0                    1000                  EPSG:4326   

                    timestamp  summary_total_sources  \
0  2025-09-19T20:37:11.082935                      4   

   summary_successful_sources  summary_total_errors  ...  \
0                           3                     1  ...   

   topography_interpreted_data_visualization_resolution_meters  \
0                                               30.0             

  topography_interpreted_data_visualization_color_scheme  \
0                                               None       

   topography_interpreted_data_risk_assessment  \
0                                          LOW   

  topography_metadata_processing_time_ms topography_metadata_data_currency  \
0                     

In [ ]:
from sklearn.tree import DecisionTreeClassifier


In [ ]:
# Add sample variation to target column
import numpy as np

# Randomly assign vegetation health states
df[target_col] = np.random.choice(['STRESSED', 'HEALTHY', 'LOW'], size=len(df))

# Rebuild the CHAID tree
tree = Tree.from_pandas_df(df, {}, target_col, max_depth=3, min_child_node_size=1)
tree.print_tree()


NameError: name 'df' is not defined